# Rysk Finance

Defi options AMM

[Docs](shttps://docs.rysk.finance/getting-started/what-is-rysk)

Rysk options are priced using a AMM.


# Trading competition

[Trading comp](https://comp.rysk.finance/) is open until June.

# Data retrieval

We can retrieve data from the Rysk finance contracts from the follwoing [subgraph](https://api.goldsky.com/api/public/project_clhf7zaco0n9j490ce421agn4/subgraphs/devey/0.0.2/gn)

There is a depth of on chain data from the subgraph.

For example, we can retrieve all of the available markets as so;

In [117]:
# install dependencies
!pip install pandas ccxt web3

You should consider upgrading via the '/home/tom/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [1]:
# we can retrieve the data using a simple client;
from dataclasses import dataclass
import requests
import json

# -H "Content-Type: application/json" -X POST -d 
@dataclass
class SubgraphClient:
    url: str
    
    
    def query(self, query):
        """Simple function to call a subgraph query."""
        headers = {
            "Content-Type": "application/json"
        }
        subgraph_query = {
            "query": query
        }
        response = requests.post(
            url=self.url,
            headers=headers,
            data=json.dumps(subgraph_query)
        )
        data = json.loads(response.content)['data']
        return data

In [2]:
subgraph_url = "https://api.goldsky.com/api/public/project_clhf7zaco0n9j490ce421agn4/subgraphs/devey/0.0.2/gn"

subgraph_query = """
{series 
  {   
    id 
    expiration 
    netDHVExposure 
    strike
    isPut
    isBuyable
    isSellable
  }
}
"""

client = SubgraphClient(subgraph_url)
 
results = client.query(subgraph_query)

results['series'][:1]



[{'id': '0x01f460be7389b109cc3599941166ea851d0b7c787badf04b1f276d3ce9269a34',
  'expiration': '1688112000',
  'netDHVExposure': '-368000000000000000000',
  'strike': '1700000000000000000000',
  'isPut': True,
  'isBuyable': True,
  'isSellable': True}]

In [21]:
# we then parse into a pandas dataframe.

import pandas as pd
from datetime import datetime

# we need to convert the timestamp of the expiration to a date, we also need to clean up the decimals.
# conveniently the options expirations align with those from deribit;
# we format to allow easier look up later


price_devisor = 1_000_000_000_000_000_000
exposure_devisor = 100_000_000_000_000_0000

def from_timestamp(date_string):
    """Parse a timestamp."""
    return datetime.fromtimestamp(int(date_string))

def to_human_format(row):
    """
    Format the row to align to the ccxt unified client.
    'ETH-16MAY23-1550-C'
    """

    month_code = row.expiration_datetime.strftime("%b").upper()
    day = row.expiration_datetime.strftime("%d")
    year = str(row.expiration_datetime.year)[2:]
    strike_price = str(row.strike_price)

    return f"ETH-{day}{month_code}{year}-{strike_price}-{'P' if row.isPut else 'C'}"


df = pd.DataFrame(results['series'])


# We format the data
df['expiration_datetime'] = df["expiration"].apply(from_timestamp)
df['strike_price'] = df['strike'].apply(lambda x: int(int(x) / price_devisor))
df['net_DHV_exposure'] = df['netDHVExposure'].apply(lambda x: int(int(x) / exposure_devisor))
df["human_strike"] = df.apply(to_human_format, axis=1)




# we filter out all markets that have already closed;
df = df[df["expiration_datetime"] > datetime.now()]



# we filter out markets that we cant trade at all
df = df[df["isBuyable"] | df['isSellable']]


# We only display our interested columns
columns = ["human_strike", "expiration_datetime", "strike_price", "net_DHV_exposure", "isPut", "isBuyable", "isSellable"]
df[columns].sort_values(["expiration_datetime", "strike_price"]).head(50)


,human_strike,expiration_datetime,strike_price,net_DHV_exposure,isPut,isBuyable,isSellable
78,ETH-19MAY23-1800-C,2023-05-19 10:00:00,1800,-115,False,True,True
79,ETH-19MAY23-1800-P,2023-05-19 10:00:00,1800,-454,True,True,True
43,ETH-19MAY23-1850-C,2023-05-19 10:00:00,1850,-387,False,True,True
82,ETH-19MAY23-1850-P,2023-05-19 10:00:00,1850,-101,True,True,True
30,ETH-26MAY23-1700-P,2023-05-26 10:00:00,1700,-525,True,True,True
40,ETH-26MAY23-1750-P,2023-05-26 10:00:00,1750,-379,True,True,True
3,ETH-26MAY23-1800-C,2023-05-26 10:00:00,1800,-222,False,True,True
31,ETH-26MAY23-1800-P,2023-05-26 10:00:00,1800,-351,True,True,True
20,ETH-26MAY23-1850-P,2023-05-26 10:00:00,1850,-236,True,True,True
33,ETH-26MAY23-1850-C,2023-05-26 10:00:00,1850,-311,False,True,True


# Exposure Arbitrages

Conceptually, the DHV is pricing its options based on demand, where by the price is a function of the expected volitity based on the BlackSholes formula combined with net exposure at each strike.

We can therefore seek to find a number of arbitrages within the markets;

We will initially demonstrate a function to identify arbitrage based on short term options versus long term options.

As a basic strategy, we will look at each side of each of the markets. For each of the tradable options, we will look for circumstances whereby the option may be immediately sold to either capitalise on the differential between the price, or the differentical between the time.


In [22]:
def does_arb_exist(row, display=False):
    """We will check the inital dataframe."""
    # find similar candidates
    candidates = df[df.isPut == row.isPut]
    
    # we filter out older markets
    candidates = candidates[candidates.expiration_datetime >= row.expiration_datetime]
    
    if row.isBuyable:
        candidates = candidates[candidates.isSellable]
    
    if row.isSellable:
        candidates = candidates[candidates.isBuyable]
    
    if row.isPut:
        # we want to look for puts which have a strike greater than our current row.
        candidates = candidates[candidates.strike >= row.strike]
        if row.isBuyable:
            # we are looking to *buy* cheap puts then sell them on.
            # expensive options are those where the dhv has has sold a lot at that strike 
            # and is therefore increasing the price to compensate. 
            # This is denoted with a negative exposure
            
            # our candidates will be those with new_dhv_exposure > our row 
            candidates = candidates[candidates.net_DHV_exposure > row.net_DHV_exposure]
            candidates.sort_values("net_DHV_exposure")
            
        else:
            # We now are looking to *sell* expensive puts 
            # and then buy cheap ones
            candidates = candidates[candidates.net_DHV_exposure < row.net_DHV_exposure]
            
    
    else:
        # we want to look for call which have a strike lower than our current row
        candidates = candidates[candidates.strike <= row.strike]
        if row.isBuyable:
            # we are looking to *buy* cheap puts then sell them on.
            # expensive puts are those where the dhv has has sold a lot at that strike 
            # and is therefore increasing the price to compensate. 
            # This is denoted with a negative exposure
            
            # our candidates will be those with new_dhv_exposure > our row 
            candidates = candidates[candidates.net_DHV_exposure > row.net_DHV_exposure]
        
        else:
            # We now are looking to *sell* expensive puts 
            # and then buy cheap ones
            candidates = candidates[candidates.net_DHV_exposure < row.net_DHV_exposure]
        
    def _format_row(row):
        """Format the row."""
        return f"{row.human_strike} @ {row.net_DHV_exposure}"
    

        
        
    if len(candidates):
        
#       # we now look for the biggest differences between the candidates and the row. 
        # we return this so we further can sort.
        
        candidates['value'] = abs(candidates.net_DHV_exposure - row.net_DHV_exposure)
        
        candidates = candidates.sort_values('value')
        if display:
            print(f"Potential Arb between: {_format_row(row)} & {_format_row(candidates.iloc[0])}")


    return False


df['arbs'] = df.apply(does_arb_exist, axis=1)
top_arbs = df.sort_values("arbs", ascending=False).head()


top_arbs.apply(does_arb_exist, axis=1, args=(True,))

Potential Arb between: ETH-30JUN23-1700-P @ -368 & ETH-30JUN23-1900-P @ -229
Potential Arb between: ETH-19MAY23-1850-C @ -387 & ETH-26MAY23-1850-C @ -311
Potential Arb between: ETH-19MAY23-1800-P @ -454 & ETH-26MAY23-1800-P @ -351


0     False
43    False
82    False
79    False
78    False
dtype: bool

# Price Retrieval 

Prices are extracted from the [Beyond Pricer](https://goerli.arbiscan.io/address/0xc939df369C0Fc240C975A6dEEEE77d87bCFaC259#readContract)

This allows us to take the id retrieved from the subgraph and use it to extract all of the prices, (Unfortunately this is iteratively, however this could be improved by implementing and deploying a smart contract which would collect the data in one call)

In [5]:
# we first need to import the contracts abi
# follow the instructions in ./third_party in order to compile contracts

with open("./contracts/BeyondPricer.sol/BeyondPricer.json", "r") as file:
    abi = json.loads(file.read())['abi']
    
    

In [6]:
from web3 import Web3


w3 = Web3(Web3.HTTPProvider('https://arbitrum-goerli.rpc.thirdweb.com'))


address = '0xc939df369C0Fc240C975A6dEEEE77d87bCFaC259'
contract_instance = w3.eth.contract(address=address, abi=abi)

# to prove we have the correct contract
contract_instance.functions.collateralAsset().call()


'0x408c5755b5c7a0a28D851558eA3636CfC5b5b19d'

In [7]:
df.iloc[0]

id                     0x01f460be7389b109cc3599941166ea851d0b7c787bad...
expiration                                                    1688112000
netDHVExposure                                    -368000000000000000000
strike                                            1700000000000000000000
isPut                                                               True
isBuyable                                                           True
isSellable                                                          True
expiration_datetime                                  2023-06-30 10:00:00
strike_price                                                        1700
net_DHV_exposure                                                    -368
human_strike                                          ETH-30JUN23-1700-P
arbs                                                               False
Name: 0, dtype: object

In [23]:
def filter_by_human_format(df, human_format):
    """
    Filter the DataFrame based on the human-readable format and return a single row.
    """

    filtered_option = df[df["human_strike"] == human_format]
    if len(filtered_option) > 0:
        return filtered_option.iloc[0]  # Return the first row if found
    else:
        return None  # Return None if no matching row is found



def get_options_prices(market, amount=1000000000000000000, side="buy", collateral="eth"):
    """, 
    We call the beyond pricer to determine the prices for a market
    huge thanks to 0xPawel2 and Jib &&
    """
    if side not in ["buy", "sell"]:
        raise ValueError("Side must be buy or sell")

    option_data = filter_by_human_format(df, market)
    if option_data is None:
        raise TypeError("Unable to find the market")

    if not Collateral.is_supported(collateral):
        raise TypeError(f"Collateral {collateral} is not supported")
    # here we call the contract functions

    option_series = (
        int(option_data['expiration']),
        int(option_data['strike']),
        bool(option_data['isPut']),
        '0x3b3a1dE07439eeb04492Fa64A889eE25A130CDd3', 
        '0x408c5755b5c7a0a28D851558eA3636CfC5b5b19d', 
        '0x408c5755b5c7a0a28D851558eA3636CfC5b5b19d'
    )

    result = contract_instance.functions.quoteOptionPrice(
        option_series,
        int(amount),
        True if side == "sell" else False,
        int(option_data['netDHVExposure'])).call()
    # totalPremium
    # totalDelta
    # totalFees
    return result[0] / 1_000_000

market = "ETH-19MAY23-1800-C"

df["ask"] = df.human_strike.apply(get_options_prices)
df['bid'] = df.human_strike.apply(get_options_prices, side="sell")

df[columns + ["ask", "bid"]].head()

,human_strike,expiration_datetime,strike_price,net_DHV_exposure,isPut,isBuyable,isSellable,ask,bid
0,ETH-30JUN23-1700-P,2023-06-30 10:00:00,1700,-368,True,True,True,206.786965,199.991448
2,ETH-30JUN23-1800-C,2023-06-30 10:00:00,1800,-255,False,True,True,275.701467,267.606442
3,ETH-26MAY23-1800-C,2023-05-26 10:00:00,1800,-222,False,True,True,117.656970,116.203176
19,ETH-30JUN23-2200-C,2023-06-30 10:00:00,2200,-569,False,True,True,149.138195,144.055916
20,ETH-26MAY23-1850-P,2023-05-26 10:00:00,1850,-236,True,True,True,127.533412,125.825562


In [31]:
# Unified crypto client.

import ccxt
# we filter out everything except ethereum options



exchange = ccxt.deribit()
markets = exchange.fetch_markets()



ticker = exchange.fetch_ticker(market)


print(f"{market}: bid: {ticker['bid']} ask: {ticker['ask']}")

ETH-19MAY23-1800-C: bid: 0.007 ask: 0.039


In [57]:
# we need a function to convert from eth to USD to align to rysk trades

price = exchange.fetch_ticker("ETH/USDC")['last']

def get_price_from_cex(market):
    """Get bid and ask from the exchange."""
    ticker = exchange.fetch_ticker(market)
    bid, ask = ticker['bid'], ticker['ask']
    
    if bid is not None:
        bid = price * bid
    if ask is not None:
        ask = price * ask
    return bid, ask
df['cex'] = df.human_strike.apply(get_price_from_cex)
df['db_bid'] = df['cex'].apply(lambda x: x[0])
df['db_ask'] = df['cex'].apply(lambda x: x[1])

df.dropna(inplace=True)



,human_strike,expiration_datetime,strike_price,net_DHV_exposure,isPut,isBuyable,isSellable,bid,ask,db_bid,db_ask
0,ETH-30JUN23-1700-P,2023-06-30 10:00:00,1700,-368,True,True,True,199.991448,206.786965,63.877450,64.789985
2,ETH-30JUN23-1800-C,2023-06-30 10:00:00,1800,-255,False,True,True,267.606442,275.701467,130.492505,133.230110
3,ETH-26MAY23-1800-C,2023-05-26 10:00:00,1800,-222,False,True,True,116.203176,117.656970,61.139845,63.877450
19,ETH-30JUN23-2200-C,2023-06-30 10:00:00,2200,-569,False,True,True,144.055916,149.138195,25.550980,26.463515
20,ETH-26MAY23-1850-P,2023-05-26 10:00:00,1850,-236,True,True,True,125.825562,127.533412,59.314775,61.139845
28,ETH-30JUN23-2100-C,2023-06-30 10:00:00,2100,-408,False,True,True,126.030036,131.465467,36.501400,38.326470
29,ETH-30JUN23-1700-C,2023-06-30 10:00:00,1700,-170,False,True,True,306.618622,315.919206,155.130950,237.259100
30,ETH-26MAY23-1700-P,2023-05-26 10:00:00,1700,-525,True,True,True,62.958469,63.779250,10.950420,11.862955
31,ETH-26MAY23-1800-P,2023-05-26 10:00:00,1800,-351,True,True,True,107.210855,108.577213,34.676330,36.501400
33,ETH-26MAY23-1850-C,2023-05-26 10:00:00,1850,-311,False,True,True,91.968871,93.136549,37.413935,38.326470


In [68]:
df['arb_gain'] = (df['bid'] - df['db_ask'] ) /  df['db_ask'] * 100

df[columns + ["bid", "db_ask", "arb_gain"]].sort_values("arb_gain", ascending=False)

,human_strike,expiration_datetime,strike_price,net_DHV_exposure,isPut,isBuyable,isSellable,bid,db_ask,arb_gain
52,ETH-30JUN23-1500-P,2023-06-30 10:00:00,1500,-557,True,True,True,140.918936,23.725910,493.945337
19,ETH-30JUN23-2200-C,2023-06-30 10:00:00,2200,-569,False,True,True,144.055916,26.463515,444.356696
30,ETH-26MAY23-1700-P,2023-05-26 10:00:00,1700,-525,True,True,True,62.958469,11.862955,430.714894
61,ETH-30JUN23-1600-P,2023-06-30 10:00:00,1600,-494,True,True,True,182.419204,40.151540,354.326793
87,ETH-26MAY23-2000-C,2023-05-26 10:00:00,2000,-487,False,True,True,38.266928,9.125350,319.347510
79,ETH-19MAY23-1800-P,2023-05-19 10:00:00,1800,-454,True,True,True,41.676240,10.950420,280.590334
55,ETH-26MAY23-1950-C,2023-05-26 10:00:00,1950,-440,False,True,True,51.651824,13.688025,277.350450
28,ETH-30JUN23-2100-C,2023-06-30 10:00:00,2100,-408,False,True,True,126.030036,38.326470,228.832882
40,ETH-26MAY23-1750-P,2023-05-26 10:00:00,1750,-379,True,True,True,67.877241,20.988305,223.405063
0,ETH-30JUN23-1700-P,2023-06-30 10:00:00,1700,-368,True,True,True,199.991448,64.789985,208.676484
